In [4]:
from datasets import load_dataset, get_dataset_config_names
import pandas as pd

/home/arafat/projects/deeplearning_crash_course/dl_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Downloading

In [166]:
config = get_dataset_config_names('Helsinki-NLP/opus-100')
data = load_dataset('Helsinki-NLP/opus-100', 'bn-en')
data['train'].to_csv("./data/train_data.csv")

Creating CSV from Arrow format: 100%|██████████| 1000/1000 [00:05<00:00, 187.98ba/s]


265553291

## Data 

In [167]:
df = pd.read_csv('/home/arafat/projects/deeplearning_crash_course/data/train_data.csv')
df['translation_dict'] = df['translation'].apply(eval)


df['bn'] = df['translation_dict'].apply(lambda x: x['bn'])
df['en'] = df['translation_dict'].apply(lambda x: x['en'])


df = df.drop('translation_dict', axis=1)
df.to_csv("./data/bangla_english_translation.csv")

In [163]:
df.loc[100:150, ['bn', 'en']].values[:, 0]

array(['চিহ্নিত অনুসন্ধানের ফোল্ডারের নাম উল্লেখ করা আবশ্যক',
       'কাফেররা যেন মনে না করে যে আমি যে , অবকাশ দান করি , তা তাদের পক ্ ষে কল ্ যাণকর । আমি তো তাদেরকে অবকাশ দেই যাতে করে তারা পাপে উন ্ নতি লাভ করতে পারে । বস ্ তুতঃ তাদের জন ্ য রয়েছে লাঞ ্ ছনাজনক শাস ্ তি ।',
       'সেদিন তাদের চক ্ রান ্ ত তাদের কোন উপকারে আসবে না এবং তারা সাহায ্ যপ ্ রাপ ্ তও হবে না ।',
       'একটা দুঃসংবাদ দিবো।',
       "আর ইহুদী ও খ ্ রীষ ্ টানরা বলে -- ''আমরা আল্লাহ্\u200cর সন্তান ও তাঁর প্রিয়পাত্র।’’ তুমি বলো -- ''তবে কেন তোমাদের অপরাধের জন্য তিনি তোমাদের শাস্তি দেন? না, যাদের তিনি সৃষ্টি করেছেন তোমরা তাদের মধ্যেকার মানুষ। তিনি যাকে ইচ্ছে করেন পরিত্রাণ করেন এবং যাকে ইচ্ছে করেন শাস্তি দেন।’’ আর আল্লাহ্\u200cরই মহাকাশমন্ডল ও পৃথিবীর সাম্রাজ্য এবং এই দুইয়ের মধ্যে যা আছে, আর তাঁরই দিকে প্রত্যাবর্তন।",
       'এটা দেখ তো।',
       'এই রসূলগণ-আমি তাদের কাউকে কারো উপর মর ্ যাদা দিয়েছি । তাদের মধ ্ যে কেউ তো হলো তারা যার সাথে আল ্ লাহ কথা বলেছেন , আর কারও মর ্ যাদা উচ ্ চতর করেছেন এবং আমি মরিয়

## Train a Tokenizer

In [1]:
import os
from typing import Literal

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

In [2]:
def train_and_save_tokenizer(file_path:str, output_path:str, vocab_size:int, lang:str = Literal['bn', 'en']):
    if not os.path.exists(file_path):
        raise FileNotFoundError("File Path Does Not Exist")
    
    df = pd.read_csv(file_path)
    df = df.dropna()
    training_data = df.loc[:, lang].values
    
    tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()
    trainer = WordLevelTrainer(vocab_size=vocab_size,
                               min_frequency=2,
                               special_tokens=["[UNK]", "[PAD]", "[EOS]", "[SOS]"])
    
    tokenizer.train_from_iterator(training_data, trainer)
    
    os.makedirs(output_path, exist_ok=True)
    tokenizer.save(f"./{output_path}/{lang}_tokenizer.json")
    return tokenizer

In [5]:
path = '/home/arafat/projects/deeplearning_crash_course/data/bangla_english_translation.csv'
en_tokenizer = train_and_save_tokenizer(path, 'Tokenizer', 1000, 'en')
bn_tokenizer = train_and_save_tokenizer(path, 'Tokenizer', 1000, 'bn')

In [14]:
tokenizer = Tokenizer.from_file('/home/arafat/projects/deeplearning_crash_course/Tokenizer/en_tokenizer.json')
tokenizer.enable_padding(
    pad_id=tokenizer.token_to_id("[PAD]"),
    pad_token="[PAD]",
    length=None,
    # length=50,
    direction='right'
)

sample_text = "This is a test sentence"
encoding = tokenizer.encode(sample_text)
encoding.ids, encoding.tokens

# token = torch.stack([encoding[0].ids, encoding[1].ids], padding=True)

([89, 14, 11, 0, 0], ['This', 'is', 'a', '[UNK]', '[UNK]'])

## DataLoader

In [68]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [193]:
def collate_function(batch):
    bn_sentences = [item[0] for item in batch]
    en_sentences = [item[1] for item in batch]
    bn_padded = torch.nn.utils.rnn.pad_sequence(bn_sentences, batch_first=True, padding_value=1)
    en_padded = torch.nn.utils.rnn.pad_sequence(en_sentences, batch_first=True, padding_value=1)
    return bn_padded, en_padded

In [ ]:
class MachineTranslation(Dataset):
    def __init__(self, file_path:str, en_tokenizer_path:str, bn_tokenizer_path:str, split:str='train'):
        super().__init__()

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File Path does not exists for {file_path}")
        if not os.path.exists(en_tokenizer_path):
            raise FileNotFoundError(f"File Path does not exists for {en_tokenizer_path}")
        if not os.path.exists(bn_tokenizer_path):
            raise FileNotFoundError(f"File Path does not exists for {bn_tokenizer_path}")
        assert split in ["train", "validation"], "Split must be either 'train' or 'validation'"
                
        self.file_path = file_path
        self.csv_file = pd.read_csv(file_path)
        if split == "train":
            self.csv_file = self.csv_file.sample(frac=0.8, random_state=42)
            
        self.en_tokenizer = Tokenizer.from_file(en_tokenizer_path)
        self.bn_tokenizer = Tokenizer.from_file(bn_tokenizer_path)
        
        self.en_tokenizer.enable_padding(
            pad_id=self.en_tokenizer.token_to_id("[PAD]"),
            pad_token="[PAD]",
            length=None,
            direction='right'
        )
        
        self.bn_tokenizer.enable_padding(
            pad_id=self.bn_tokenizer.token_to_id("[PAD]"),
            pad_token="[PAD]",
            length=None,
            direction='right'
        )
        
        
    def __getitem__(self, idx):
        sentences = self.csv_file.loc[idx, ['bn', 'en']].values
        bangla, english = sentences[0], sentences[1]
        bn_encoding = self.bn_tokenizer.encode(bangla)
        en_encoding = self.en_tokenizer.encode(english)
        return torch.tensor(bn_encoding.ids), torch.tensor(en_encoding.ids)
        
    
    def __len__(self):
        return len(self.csv_file)

In [195]:
dataset = MachineTranslation(path, './Tokenizer/en_tokenizer.json', './Tokenizer/bn_tokenizer.json')
dataloader = DataLoader(dataset, batch_size=10, shuffle=True, collate_fn=collate_function)

In [202]:
x, y = next(iter(dataloader))
x.shape, y.shape

(torch.Size([10, 57]), torch.Size([10, 67]))

In [212]:
token_id = x[9].tolist()
bn_tokenizer.decode(token_id)


'ছবি জন্য তৈরি করা'

In [215]:
df = pd.read_csv('/home/arafat/projects/deeplearning_crash_course/data/bangla_english_translation.csv')

bn    হ্যাঁ?
en     Yeah?
Name: 0, dtype: object

In [217]:
df.loc[100:120, ['bn', 'en']]

,bn,en
100,চিহ্নিত অনুসন্ধানের ফোল্ডারের নাম উল্লেখ করা আ...,You must name this Search Folder.
101,"কাফেররা যেন মনে না করে যে আমি যে , অবকাশ দান ক...",Those who disbelieve should not assume that We...
102,সেদিন তাদের চক ্ রান ্ ত তাদের কোন উপকারে আসব...,A day on which their scheming will not benefit...
103,একটা দুঃসংবাদ দিবো।,I've got some bad news.
104,আর ইহুদী ও খ ্ রীষ ্ টানরা বলে -- ''আমরা আল্লা...,"No : You are only mortals , of His creation . ..."
105,এটা দেখ তো।,Check that out.
106,এই রসূলগণ-আমি তাদের কাউকে কারো উপর মর ্ যাদা ...,"And to Jesus , son of Mary , We gave tokens , ..."
107,"দুর ্ বলরা অহংকারীদেরকে বলবে , বরং তোমরাই তো...",Those who were despised say unto those who wer...
108,রুহিতনের বিবি,the queen of diamonds
109,"এখন বাইরে চলো, চলো, যাই","Come on, let's go."


In [23]:
x = torch.tensor([1,2,3,4,10])
y = torch.tensor([2,3,4,5])
torch.cat([x, y])


tensor([ 1,  2,  3,  4, 10,  2,  3,  4,  5])

In [1]:
import torch

In [6]:
mask = torch.zeros(10, 10)
mask[:x.shape[0], :x.shape[1]] = 1
mask

tensor([[1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
x = torch.rand(5,5)


tensor([[0.7036, 0.4407, 0.7141, 0.2086, 0.4266],
        [0.8826, 0.2820, 0.4906, 0.6313, 0.6343],
        [0.2236, 0.9251, 0.9680, 0.4137, 0.7161],
        [0.2739, 0.2671, 0.5580, 0.2215, 0.4544],
        [0.6278, 0.4673, 0.2455, 0.0232, 0.7803]])

In [8]:
import pandas as pd
data = pd.read_csv('D:/notebook/deeplearning_crash_course/data/bangla_english_translation.csv')

In [9]:
sample = data.sample(frac=0.3)
other = data[data.index != sample.index]


ValueError: Lengths must match to compare